In [92]:
import pandas as pd
import numpy as np
pd.set_option("mode.copy_on_write", True)

In [14]:
import getpass
pg_password = getpass.getpass()

 ········


In [23]:
import psycopg2
pg_dsn = f"dbname=replicator user=replicator host=127.0.0.1 port=6541 password={pg_password}"
tmp_sql = "select now()"
with psycopg2.connect(pg_dsn) as conn:
    with conn.cursor() as cursor:
        cursor.execute(tmp_sql)
        print(cursor.fetchone())

(datetime.datetime(2024, 3, 9, 19, 29, 26, 504660, tzinfo=datetime.timezone.utc),)


In [198]:
import tempfile
from collections import defaultdict

def ijv_df_read_sql_tmpfile(query):
    with tempfile.TemporaryFile() as tmpfile:
        copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
           query=query, head="HEADER"
        )
        with psycopg2.connect(pg_dsn) as conn:
            with conn.cursor() as cursor:
                cursor.copy_expert(copy_sql, tmpfile)
                tmpfile.seek(0)
                # types = defaultdict(np.uint64, i='Int32', j='Int32')
                df = pd.read_csv(tmpfile, dtype={'i':'Int32', 'j': 'Int32'})
                return df

In [199]:
%%time
likes_sql = """
    SELECT fid as i, target_cast_fid as j, count(1) as likes_v 
    FROM reactions 
    WHERE type=1
    AND fid < 10000
    GROUP BY i, j
    -- limit 100
  """
likes_df = ijv_df_read_sql_tmpfile(likes_sql)

CPU times: user 550 ms, sys: 76.6 ms, total: 627 ms
Wall time: 4.91 s


In [200]:
likes_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271944 entries, 0 to 271943
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   i        271944 non-null  Int32
 1   j        271930 non-null  Int32
 2   likes_v  271944 non-null  int64
dtypes: Int32(2), int64(1)
memory usage: 4.7 MB


In [201]:
likes_df.sample(5)

,i,j,likes_v
191239,2056,14487,1
73229,3792,548,2
213509,4286,12443,1
119797,9265,1407,2
190970,2056,2770,1


In [202]:
likes_df.index.dtype

dtype('int64')

In [203]:
%%time
replies_sql = """
    SELECT fid as i, parent_fid as j, count(1) as replies_v 
    FROM casts
    WHERE parent_hash IS NOT NULL
    AND fid < 10000
    GROUP by i, j
  """
replies_df = ijv_df_read_sql_tmpfile(replies_sql)

CPU times: user 463 ms, sys: 61.2 ms, total: 524 ms
Wall time: 4.64 s


In [204]:
replies_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231879 entries, 0 to 231878
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   i          231879 non-null  Int32
 1   j          231879 non-null  Int32
 2   replies_v  231879 non-null  int64
dtypes: Int32(2), int64(1)
memory usage: 4.0 MB


In [205]:
%%time
mentions_sql = """
		WITH mention AS (
			SELECT fid as author_fid, mention.value as mention_fid 
			FROM casts, json_array_elements_text(casts.mentions) as mention
		)
		SELECT 
			author_fid as i, mention_fid as j, count(1) as mentions_v
		FROM mention
        WHERE author_fid < 10000
		GROUP BY i, j
  """
mentions_df = ijv_df_read_sql_tmpfile(mentions_sql)

CPU times: user 108 ms, sys: 18.6 ms, total: 126 ms
Wall time: 2.65 s


In [206]:
mentions_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48679 entries, 0 to 48678
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   i           48679 non-null  Int32
 1   j           48679 non-null  Int32
 2   mentions_v  48679 non-null  int64
dtypes: Int32(2), int64(1)
memory usage: 855.8 KB


In [207]:
%%time
recasts_sql = """
    SELECT fid as i, target_cast_fid as j, count(1) as reacts_v 
    FROM reactions 
    WHERE type=2
    AND fid < 10000
    GROUP BY i, j
  """
recasts_df = ijv_df_read_sql_tmpfile(recasts_sql)

CPU times: user 102 ms, sys: 23 ms, total: 125 ms
Wall time: 2.16 s


In [208]:
recasts_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43143 entries, 0 to 43142
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   i         43143 non-null  Int32
 1   j         43139 non-null  Int32
 2   reacts_v  43143 non-null  int64
dtypes: Int32(2), int64(1)
memory usage: 758.5 KB


In [209]:
%%time
follows_sql = """
    SELECT 
        follower_fid as i, 
        following_fid as j,
        1 as follows_v
    FROM mv_follow_links 
    WHERE follower_fid < 10000
    ORDER BY i, j, follows_v desc
    """
follows_df = ijv_df_read_sql_tmpfile(follows_sql)

CPU times: user 2.05 s, sys: 210 ms, total: 2.26 s
Wall time: 13.3 s


In [210]:
follows_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100115 entries, 0 to 1100114
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   i          1100115 non-null  Int32
 1   j          1100115 non-null  Int32
 2   follows_v  1100115 non-null  int64
dtypes: Int32(2), int64(1)
memory usage: 18.9 MB


In [211]:
%%time
df = likes_df.merge(replies_df, how='outer', left_on=['i','j'], right_on=['i','j'], indicator=False)

CPU times: user 120 ms, sys: 34.1 ms, total: 155 ms
Wall time: 166 ms


In [212]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403229 entries, 0 to 403228
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   i          403229 non-null  Int32  
 1   j          403215 non-null  Int32  
 2   likes_v    271944 non-null  float64
 3   replies_v  231879 non-null  float64
dtypes: Int32(2), float64(2)
memory usage: 10.0 MB


In [213]:
%%time
df = df.merge(mentions_df, how='outer', left_on=['i','j'], right_on=['i','j'], indicator=False)

CPU times: user 116 ms, sys: 36.1 ms, total: 152 ms
Wall time: 159 ms


In [214]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421416 entries, 0 to 421415
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   i           421416 non-null  Int32  
 1   j           421402 non-null  Int32  
 2   likes_v     271944 non-null  float64
 3   replies_v   231879 non-null  float64
 4   mentions_v  48679 non-null   float64
dtypes: Int32(2), float64(3)
memory usage: 13.7 MB


In [215]:
%%time
df = df.merge(recasts_df, how='outer', left_on=['i','j'], right_on=['i','j'], indicator=False)

CPU times: user 98.7 ms, sys: 32.7 ms, total: 131 ms
Wall time: 136 ms


In [216]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424852 entries, 0 to 424851
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   i           424852 non-null  Int32  
 1   j           424837 non-null  Int32  
 2   likes_v     271944 non-null  float64
 3   replies_v   231879 non-null  float64
 4   mentions_v  48679 non-null   float64
 5   reacts_v    43143 non-null   float64
dtypes: Int32(2), float64(4)
memory usage: 17.0 MB


In [217]:
%%time
df = df.merge(follows_df, how='outer', left_on=['i','j'], right_on=['i','j'], indicator=False)

CPU times: user 404 ms, sys: 103 ms, total: 506 ms
Wall time: 586 ms


In [218]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303353 entries, 0 to 1303352
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   i           1303353 non-null  Int32  
 1   j           1303338 non-null  Int32  
 2   likes_v     271944 non-null   float64
 3   replies_v   231879 non-null   float64
 4   mentions_v  48679 non-null    float64
 5   reacts_v    43143 non-null    float64
 6   follows_v   1100115 non-null  float64
dtypes: Int32(2), float64(5)
memory usage: 62.1 MB


In [219]:
df.sample(10)

,i,j,likes_v,replies_v,mentions_v,reacts_v,follows_v
1246328,9595,731,NaN,NaN,NaN,NaN,1.0
1062523,7005,134,NaN,NaN,NaN,NaN,1.0
1219019,9143,23798,NaN,NaN,NaN,NaN,1.0
1227063,9228,12540,NaN,NaN,NaN,NaN,1.0
566870,880,465,NaN,NaN,NaN,NaN,1.0
1152723,8327,8296,NaN,NaN,NaN,NaN,1.0
1228254,9230,451,NaN,NaN,NaN,NaN,1.0
710993,2182,1982,NaN,NaN,NaN,NaN,1.0
316442,3278,3278,NaN,1.0,NaN,NaN,NaN
412918,3647,1495,NaN,NaN,2.0,NaN,1.0


In [228]:
df['l1rep1rec1m1'] = df.loc[:,['likes_v', 'replies_v', 'mentions_v', 'reacts_v', 'follows_v']].sum(axis=1)

In [236]:
# Very Slow code
# df['l1rep6rec3m12'] = df.apply(lambda x: x['likes_v'] + 
#                                            x['replies_v'] * 6.0 +
#                                            x['reacts_v'] * 3.0 + 
#                                            x['mentions_v'] * 12.0 +
#                                            x['follows_v'], 
#                                axis=1)

In [237]:
df['l1rep6rec3m12'] = df['likes_v'].fillna(0) \
                        + (df['replies_v'].fillna(0) * 6.0) \
                        + (df['reacts_v'].fillna(0) * 3.0) \
                        + (df['mentions_v'].fillna(0) * 12.0) \
                        + df['follows_v'].fillna(0)

In [238]:
df.sample(10)

,i,j,likes_v,replies_v,mentions_v,reacts_v,follows_v,l1rep1rec1m1,l1rep6rec3m12
255072,707,849,2.0,NaN,NaN,NaN,1.0,3.0,3.0
544629,666,752,NaN,NaN,NaN,NaN,1.0,1.0,1.0
458528,243,293,NaN,NaN,NaN,NaN,1.0,1.0,1.0
21389,559,9546,1.0,NaN,NaN,NaN,1.0,2.0,2.0
541906,649,613,NaN,NaN,NaN,NaN,1.0,1.0,1.0
319199,518,371,NaN,1.0,NaN,NaN,NaN,1.0,6.0
1049857,6787,358,NaN,NaN,NaN,NaN,1.0,1.0,1.0
186724,306,5701,1.0,7.0,3.0,NaN,1.0,12.0,80.0
1188330,8831,18830,NaN,NaN,NaN,NaN,1.0,1.0,1.0
934192,5033,880,NaN,NaN,NaN,NaN,1.0,1.0,1.0


In [239]:
lt_l1rep6rec3m12enhanced_df = df[df['follows_v'].notna()][['i','j','l1rep6rec3m12']].rename(columns={'l1rep6rec3m12':'v'})

In [240]:
lt_l1rep6rec3m12enhanced_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1100115 entries, 1 to 1303352
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   i       1100115 non-null  Int32  
 1   j       1100115 non-null  Int32  
 2   v       1100115 non-null  float64
dtypes: Int32(2), float64(1)
memory usage: 27.3 MB


In [241]:
lt_l1rep6rec3m12enhanced_df.sample(5)

,i,j,v
902922,4553,207,1.0
782720,2898,207,1.0
162513,5925,2728,9.0
79611,312,14195,2.0
1108478,7692,11637,1.0


In [242]:
%%time
localtrust = lt_l1rep6rec3m12enhanced_df.to_dict(orient="records")

CPU times: user 2.2 s, sys: 183 ms, total: 2.38 s
Wall time: 2.56 s


In [249]:
max_lt_id = max(lt_l1rep6rec3m12enhanced_df['i'].max(), lt_l1rep6rec3m12enhanced_df['j'].max())
max_lt_id

272350

In [244]:
import random

In [248]:
random.sample(localtrust, 10)

[{'i': 8680, 'j': 6815, 'v': 1.0},
 {'i': 534, 'j': 1214, 'v': 20.0},
 {'i': 7742, 'j': 539, 'v': 1.0},
 {'i': 8432, 'j': 6227, 'v': 22.0},
 {'i': 3344, 'j': 5650, 'v': 1.0},
 {'i': 1082, 'j': 158, 'v': 1.0},
 {'i': 225, 'j': 2455, 'v': 1.0},
 {'i': 974, 'j': 884, 'v': 1.0},
 {'i': 1190, 'j': 4269, 'v': 1.0},
 {'i': 5399, 'j': 15617, 'v': 1.0}]

In [221]:
lt_follows_df = df[['i','j','follows_v']].rename(columns={'follows_v':'v'})

In [222]:
lt_follows_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303353 entries, 0 to 1303352
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   i       1303353 non-null  Int32  
 1   j       1303338 non-null  Int32  
 2   v       1100115 non-null  float64
dtypes: Int32(2), float64(1)
memory usage: 22.4 MB


In [223]:
lt_follows_df.sample(5)

,i,j,v
1115900,7781,1580,1.0
190050,9051,187822,1.0
538387,629,359,1.0
636388,1464,83,1.0
471651,330,311,1.0
